In [11]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
spark.version

'3.5.1'

In [4]:
df=spark.read.parquet('./yellow_tripdata_2024-10.parquet')

In [5]:
df.limit(10).show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [8]:
df=df.repartition(4)

In [10]:
df.write.save('./yellow_tripdata_2024-10_partitioned.parquet')

In [17]:
df.filter((col('tpep_pickup_datetime').cast('date')=='2024-10-15') & (col('tpep_dropoff_datetime').cast('date')=='2024-10-15')).count()

127993

In [20]:
df_trip=df.withColumn('duration',(unix_timestamp('tpep_dropoff_datetime')-unix_timestamp('tpep_pickup_datetime'))/3600)

In [24]:
# df_trip.limit(10).show()
df_trip.agg(max('duration')).show()

+------------------+
|     max(duration)|
+------------------+
|162.61777777777777|
+------------------+



In [33]:
zone_Lookup=spark.read.csv('./taxi_zone_lookup.csv',header=True)
zone_Lookup.createOrReplaceTempView('zone_Lookup')

In [34]:

spark.sql('select * from zone_lookup').show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [35]:
df_joined=df.join(zone_Lookup,col('PULocationID')==col('LocationID'))

In [44]:
df_grouped=df_joined.groupBy('zone').count().sort('count',truncate=False)
df_grouped.show()
# .show()

+--------------------+-----+
|                zone|count|
+--------------------+-----+
|Governor's Island...|    1|
|       Rikers Island|    2|
|       Arden Heights|    2|
|         Jamaica Bay|    3|
| Green-Wood Cemetery|    3|
|   Rossville/Woodrow|    4|
|       West Brighton|    4|
|       Port Richmond|    4|
|Eltingville/Annad...|    4|
|Charleston/Totten...|    4|
|         Great Kills|    6|
|        Crotona Park|    6|
|     Mariners Harbor|    7|
|Heartland Village...|    7|
|Saint George/New ...|    9|
|             Oakwood|    9|
|New Dorp/Midland ...|   10|
|       Broad Channel|   10|
|         Westerleigh|   12|
|     Pelham Bay Park|   12|
+--------------------+-----+
only showing top 20 rows

